In [136]:
using NCDatasets
using GRIBDatasets

###First need to determine how many times the nyquist frequency the velocity is. If it's <=1, don't need to do anything. 
###Odd intervals will constitute a fold. For example -> 
###1x nyquist frequency with remainder of n will fold to negative nyquist + remainder. 
###2x nyquist frequency with remainder of n will fold to negative nyquist + nyquist + remainder. 
###3x nyquist frequency with remainder of n will fold to negative nyquist + remainder 

###ODD times nyquist freq = (Opposite sign(v) * nyquist) + (sign(v) * remainder)
###EVEN times nyquist freq = (opposite sign(v) * nyquist) + (sign(v) * (nyquist + remainder))



###Calculate the number of folds for each gate in the wind field. 
function calc_folded(v, nyquist = nyquist_freq)

    if ismissing(v)
        return(missing)
    end 
    if abs(v) <= nyquist_freq
        return v
    else 
        ##Folded 
        rem = abs(v) % nyquist 
        vsign = sign(v) 
        ###Even number of folds 
        if ((v ÷ nyquist_freq) % 2) == 0
            v_folded = (-1 * vsign * nyquist) + (vsign * (rem + nyquist))
            return(v_folded)
        else 
            ###Odd number of multiples of nyquist freq 
            v_folded = (-1 * vsign * nyquist) + (vsign * rem)
            return(v_folded)
        end 
    end 
end 
 
###For HAFS data, vertical level options are ["isobaricInhPa", "isobaricLayer", "meanSea", "heightAboveGroundLayer", "surface", "nominalTop", "tropopause"]
##Write the fields out to a netcdf file 
###Important assumption here - variables are indexed as lat x lon x level x time 
###Some new modifications - can pass it a vector of vectors for level and a vector of level names 
function write_to_netcdf(outfile::String, input_set::String, fields::Vector{String}, field_levels::Vector{String}, levels::Vector, fold_uvw = true )


    ds = GRIBDataset(input_set, filter_by_values=Dict("typeOfLevel" => field_levels[1]))

    ##This is gridded data so we'll assume that we will always be on the same lat/lon grid 
    lat = ds["lat"][:]
    lon = ds["lon"][:]
    t = ds["valid_time"][:]
    ###If output file already exists, remove it such that we might overwrite it 
    isfile(outfile) ? rm(outfile) : ""

    ###Do some dimension definitions 
    output = Dataset(outfile, "c")
    defDim(output, "time", length(t))
    defVar(output, "time", t, ("time", ))
    defDim(output, "lat", length(lat))
    defVar(output, "lat", lat, ("lat",))
    defDim(output, "lon", length(lon))
    defVar(output, "lon", lon, ("lon", ))


    ###There will be different types of levels 
    for levtype in Set(field_levels)

        if levtype == "atmosphereSingleLayer"
            continue 
        end 

        ds = GRIBDataset(input_set, filter_by_values=Dict("typeOfLevel" => levtype))
        currlevs = ds[levtype][:]
        defDim(output, levtype, length(currlevs))
        defVar(output, levtype, currlevs, (levtype,))
    end 

    ###Grab the data for each individual field, replacing the levels that they're not defined on with arrays of missings 
    for (i, field) in enumerate(fields)

        ds = GRIBDataset(input_set, filter_by_values=Dict("typeOfLevel" => field_levels[i]))

        if field_levels[i] == "atmosphereSingleLayer"

            currv = ds[field][:,:,:]
            defVar(output, String(field), currv, ("lon", "lat", "time"))

        else 
            
           
            curr_levels = ds[field_levels[i]][:]
            if typeof(levels[1]) != Colon
                lev_idxer = [lev in levels[i] for lev in curr_levels]
            else 
                lev_idxer = [true for lev in curr_levels]
            end 
        
            t = ds["valid_time"]

            cf = ds[field]

            final_arr = Array{Union{Missing, Float64}}(fill(missing, size(cf)))
            final_arr[:,:, lev_idxer, :] .= cf[:, :, lev_idxer, :]

            defVar(output, String(field), final_arr, ("lon", "lat", field_levels[i], "time"), attrib = cf.attrib)

            if (field in ["u", "v", "w"]) & fold_uvw
                folded_arr =  map(calc_folded, final_arr)
                defVar(output, string(field) * "_FOLDED", folded_arr, ("lon", "lat", field_levels[i], "time"), 
                                attrib = Dict("name" =>  string(field) * " Velocity Folded", 
                                "longname" => string(field) * " velocity with folding applied at a nyquist frequency of " * string(nyquist_freq)))
            end 

        end 

        close(ds)
    end 
    
    close(output)
end 

write_to_netcdf (generic function with 3 methods)

<h1> This notebook contains a short code for opening HAFS model output in grib format, selecting variables, applying aliasing, and outputting to a netCDF file. </h1>

---

<h2> Converting from GRIB to netCDF </h2>

Converting from GRIB to netCDF is fairly simple. The user must simply specify an input GRIB file, (`infile`), a 
file path to output the resultant netCDF data to (`outfile`), the fields to grab from the input file (`fields`), 
the names of the vertical levels on which these fields are defined (`level_names`), and the subset of levels on which 
they are defined to write to the netcdf (`field_levels`). In the example code below, the user wishes to read in data from 
`"./HAFS_DATA/07l.2022091806.hfsb.parent.atm.f012.grb2"`, and write to `"test_out.nc"`. The variables of interest are `"u"`, 
which is defined on `"isobaricInhPa"` levels, and `"refc"`, defined only on `"atmosphereSingleLayer"` levels. One other thing to note here - 
`u` in the result dataset will have dimensions of `lon x lat x level x time`, while `refc` just of `lon x lat x time` since it's only defined on a single level. 
Finally, since `field_levels = [:, :]`, the user wishes to retrieve all available levels for both fields. 
    
---

<h2> Applying aliasing to velocity data </h2>

Other functionality provided by the code is some simple aliasing. By setting the global variable `nyquist_freq`, the user is able to modify the threshold at which 
velocity data will be folded. The default value of 22 m/s was chosen from documentation on the APEX field campaign with the NOAA Tail Doppler Radar (TDR). When the argument 
`fold_uvw` to `write_to_netcdf` is set to `true`, if any of `"u", "v", or "w"` are fields chosen to be written to the output netCDF, another version, for example, `"u_FOLDED"` will also be written with folding applied as constrained by `nyquist_freq`. 


In the simplest case, if one simply wants to calculate the aliased version of a velocity for a single measurement, simply invoke 
`calc_folded(v, nyquist = nyquist_freq)`, where `v` is the velocity in m/s and `nyquist_freq` is the nyquist frequency of the observing platform in m/s. The aliased value for 
`v` will be returned. 
---


In [138]:
###From APEX field program report -  Actual velocities measured by the TDR are folded/aliased within the Nyquist interval, 
####which is approximately -22 to 22 m/s for the standard 2775 Hz PRF used during HRD missions.
nyquist_freq = 22 ##(m/s)

ield_levels = [:, :]
fields = ["u", "refc"]
level_names = ["isobaricInhPa", "atmosphereSingleLayer"]
outfile = "test_v2_out.nc"
infile = "./HAFS_DATA/07l.2022091806.hfsb.parent.atm.f012.grb2"

"./HAFS_DATA/07l.2022091806.hfsb.parent.atm.f012.grb2"

In [ ]:
write_to_netcdf(outfile, infile, fields, level_names, field_levels, fold_uvw = true)